In [33]:
import pandas as pd
import pickle as pkl
from collections import deque
import time
from blender import BlendER, Table, SqlOrderBy
import blender.resolution_function as resolution_function
import networkx as nx
from itertools import combinations
import numpy as np

In [34]:
# Change to raw dataset path
data = pd.read_csv("datasets/alaska_cameras_small/dataset.csv")

# Change to matches (dataframe with l_id and r_id pairs)
matches = pd.read_csv("datasets/alaska_cameras_small/matches.csv")

# Change to a pickled candidate pairs file (list of pairs)
with open(
    "datasets/alaska_cameras_small/blocking_functions/candidates_sparker.pkl",
    "rb",
) as f:
    candidates = pkl.load(f)

In [35]:
# If your matches aren't using l_id and r_id, or you have a different matching function change here
def matcher(l, r):
    global count
    return (
        ((matches["l_id"] == l) & (matches["r_id"] == r))
        | ((matches["l_id"] == r) & (matches["r_id"] == l))
    ).any()

In [36]:
# Change based on the query and resolution functions you specified on blender, for accurate comparison
def resolve(records):
    entity = {}
    entity["_id"] = records['_id'].mode()[0]
    entity["description"] = records['description'].mode()[0]
    entity["brand"] = records['brand'].mode()[0]
    entity["price"] = records["price"].min()
    entity["mp"] = records["mp"].mean()
    return entity

In [37]:
def match(candidate_pairs):
    seen = set()
    matches_set = set()
    not_matches_set = set()
    result = []
    comparisons = 0
    for idx, row in data.iterrows():
        # Change if your id column is different
        root = row["_id"]
        if root in seen:
            continue

        pairs = [pair for pair in candidate_pairs if root in pair]
        if not pairs:
            continue

        block = set().union(*pairs)

        entity_cluster = set([root])
        to_analyze = deque([root])

        while to_analyze:
            id = to_analyze.popleft()
            for candidate in block:
                if candidate in entity_cluster:
                    continue
                if id == candidate:
                    entity_cluster.add(candidate)
                    continue
                if (id, candidate) in matches_set:
                    entity_cluster.add(candidate)
                    to_analyze.append(candidate)
                    continue
                if (id, candidate) in not_matches_set:
                    continue
                comparisons += 1
                if matcher(id, candidate):
                    entity_cluster.add(candidate)
                    matches_set.add((id, candidate))
                    matches_set.add((candidate, id))
                    to_analyze.append(candidate)
                else:
                    not_matches_set.add((id, candidate))
                    not_matches_set.add((candidate, id))

        seen.update(entity_cluster)
        result.append(entity_cluster)
    return result, comparisons


In [38]:
start = time.time()

results, batch_comparisons = match(candidates)

batch_elapsed_time = time.time() - start

In [39]:
# Change if your id column is different
resolved = pd.DataFrame([resolve(data[data["_id"].isin(entity)]) for entity in results])
# Change to the where query you used on blender, and the order by
batch_filtered = resolved[resolved["brand"] == "sony"].sort_values(
    "mp", ascending=False
)
batch_filtered_pairs = set()
for group in (results[index] for index in batch_filtered.index):
    if len(group) > 1:
        batch_filtered_pairs.update([tuple(sorted(pair)) for pair in combinations(group, 2)])

In [40]:
batch_filtered_pairs

{('www.ebay.com//54541', 'www.walmart.com//767'),
 ('www.ebay.com//42559', 'www.ebay.com//52701'),
 ('www.ebay.com//47096', 'www.ebay.com//52701'),
 ('www.ebay.com//23894', 'www.ebay.com//53210'),
 ('www.ebay.com//43825', 'www.ebay.com//55468'),
 ('www.ebay.com//54801', 'www.ebay.com//56504'),
 ('www.ebay.com//46019', 'www.ebay.com//55665'),
 ('www.ebay.com//53640', 'www.ebay.com//56042'),
 ('www.ebay.com//43524', 'www.ebay.com//56887'),
 ('www.ebay.com//47149', 'www.ebay.com//48157'),
 ('www.ebay.com//46233', 'www.ebay.com//56737'),
 ('www.ebay.com//45085', 'www.ebay.com//52948'),
 ('www.ebay.com//48365', 'www.ebay.com//60783'),
 ('www.ebay.com//55759', 'www.gosale.com//1044'),
 ('www.ebay.com//46180', 'www.ebay.com//47382'),
 ('www.buzzillions.com//1914', 'www.ebay.com//44928'),
 ('www.ebay.com//44366', 'www.ebay.com//52614'),
 ('www.ebay.com//53225', 'www.ebay.com//59501'),
 ('www.ebay.com//46633', 'www.ebay.com//53466'),
 ('www.ebay.com//45789', 'www.gosale.com//630'),
 ('www.ebay.

In [41]:
print("Batch comparisons:", batch_comparisons)
print("Batch elapsed time:", batch_elapsed_time)
print("Batch emitted:", len(batch_filtered))

Batch comparisons: 22073
Batch elapsed time: 59.704665660858154
Batch emitted: 206


In [42]:
# Change based on you matches file
def matcher_table(l, r):
    return (
        ((matches["l_id"] == l["table__id"]) & (matches["r_id"] == r["table__id"]))
        | ((matches["l_id"] == r["table__id"]) & (matches["r_id"] == l["table__id"]))
    ).any()

In [54]:
blender_start = None
blender_results = []
def listener(entity, cluster, comparisons):
    global blender_start       

    i = len(blender_results)

    if len(cluster) > 1:
        cluster_pairs = {tuple(sorted(pair)) for pair in combinations(cluster, 2)}
    else:
        cluster_pairs = set()

    # Change based on the attribute you used to sort
    blender_results.append({
        "elapsed_time": time.time() - blender_start,
        "comparisons": comparisons,
        "order": entity["table_mp"],
        "tp": len(cluster_pairs & batch_filtered_pairs),
        "fp": len(cluster_pairs - batch_filtered_pairs),
    })

In [55]:
G = nx.Graph()
G.add_edges_from(candidates)
blocks = [list(set(x)) for x in nx.connected_components(G)]

In [56]:
# Configure
blender = (BlendER()
    .from_table(Table(data, blocks, matcher_table, "table", "_id"))
    .select(
        ("table._id", resolution_function.VOTE),
        ("table.description", resolution_function.VOTE),
        ("table.brand", resolution_function.VOTE),
        ("table.price", resolution_function.MIN),
        ("table.mp", resolution_function.AVG),
        order_by=("table.mp", SqlOrderBy.DESC),
    )
    .where('table.brand = "sony"')
    .subscribe(listener)
)

In [57]:
blender_start = time.time()
blender_results = []
blender.run()

In [61]:
# Create evaluation dataframe
blender_dataframe = pd.DataFrame(blender_results)

if len(blender_dataframe) > 0:
    prev = blender_dataframe["order"].shift(1)
    # CHANGE < TO > IF ASC
    blender_dataframe["correct_order"] = (blender_dataframe["order"] < prev) | np.isclose(blender_dataframe["order"], prev, 1e-8)
    blender_dataframe.loc[0, "correct_order"] = True

    blender_dataframe["running_tp"] = blender_dataframe["tp"].cumsum()
    blender_dataframe["running_fp"] = blender_dataframe["fp"].cumsum()
    blender_dataframe["running_fn"] = len(batch_filtered_pairs) - blender_dataframe["running_tp"]
    blender_dataframe["running_correctness"] = blender_dataframe["correct_order"].cumsum()

    blender_dataframe["recall"] = (blender_dataframe["running_tp"] / (blender_dataframe["running_tp"] + blender_dataframe["running_fn"])).replace(np.nan, 0)
    blender_dataframe["precision"] = (blender_dataframe["running_tp"] / (blender_dataframe["running_tp"] + blender_dataframe["running_fp"])).replace(np.nan, 0)
    blender_dataframe["correctness"] = blender_dataframe["running_correctness"] / (blender_dataframe.index + 1)
    blender_dataframe["f1"] = (
        2
        * (blender_dataframe["precision"] * blender_dataframe["recall"])
        / (blender_dataframe["precision"] + blender_dataframe["recall"])
    )

    blender_dataframe.to_csv("output/blender_results.csv", index=False)
else:
    print("No results from blender to evaluate.")

In [62]:
blender_dataframe

,elapsed_time,comparisons,order,tp,fp,correct_order,running_tp,running_fp,running_fn,running_correctness,recall,precision,correctness,f1
0,1.042556,3,36.4,3,0,True,3,0,3359,1,0.000892,1.0,1.0,0.001783
1,1.178874,45,24.3,253,0,True,256,0,3106,2,0.076145,1.0,1.0,0.141515
2,1.284934,75,24.3,55,0,True,311,0,3051,3,0.092504,1.0,1.0,0.169344
3,1.339115,95,24.3,0,0,True,311,0,3051,4,0.092504,1.0,1.0,0.169344
4,1.390909,115,24.3,0,0,True,311,0,3051,5,0.092504,1.0,1.0,0.169344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,31.166019,12631,2.0,0,0,True,3362,0,0,202,1.000000,1.0,1.0,1.000000
202,31.175422,12638,2.0,0,0,True,3362,0,0,203,1.000000,1.0,1.0,1.000000
203,31.180902,12645,2.0,0,0,True,3362,0,0,204,1.000000,1.0,1.0,1.000000
204,31.184640,12652,2.0,0,0,True,3362,0,0,205,1.000000,1.0,1.0,1.000000
